<a href="https://colab.research.google.com/github/Keoni808/NFL_Data_Cleaning/blob/main/NFL_Plays_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mounting and Imports

In [ ]:
# Mount your Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
# imports

# Data manipulation
import pandas as pd

# Regular expressions
import re

# Grab data from database
from google.cloud import bigquery

# Loading Data


In [ ]:
# Client connect to bigquery project
client = bigquery.Client('nfl-data-430702')

## All Plays 2023

In [ ]:
nfl_2023_plays_query = """
                       SELECT *
                       FROM `nfl-data-430702.NFL_Scores.NFL-Plays-2023`
                       """

# Run the query, and return a pandas DataFrame
dry_run_config = bigquery.QueryJobConfig(dry_run=True)
dry_run_query = client.query(nfl_2023_plays_query, job_config=dry_run_config)
print("This query will process {} bytes.".format(dry_run_query.total_bytes_processed))

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**9)
safe_config_query = client.query(nfl_2023_plays_query, job_config=safe_config)

# df_nfl_scores_data = safe_config_query.to_dataframe()

This query will process 11827487 bytes.


In [ ]:
df_2023_plays = safe_config_query.to_dataframe()

In [ ]:
df_2023_plays.head()

##Super Bowl Plays 2023

In [ ]:
nfl_2023_sb_plays_query = """
                          SELECT *
                          FROM `nfl-data-430702.NFL_Scores.NFL-Plays-SuperBowl-2023`
                          """

# Run the query, and return a pandas DataFrame
dry_run_config = bigquery.QueryJobConfig(dry_run=True)
dry_run_query = client.query(nfl_2023_sb_plays_query, job_config=dry_run_config)
print("This query will process {} bytes.".format(dry_run_query.total_bytes_processed))

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**9)
safe_config_query = client.query(nfl_2023_sb_plays_query, job_config=safe_config)

# df_nfl_scores_data = safe_config_query.to_dataframe()

This query will process 41291 bytes.


In [ ]:
df_2023_plays_sb = safe_config_query.to_dataframe()

In [ ]:
df_2023_plays_sb.head()

,Season,Week,Day,Date,AwayTeam,HomeTeam,Quarter,DriveNumber,TeamWithPossession,IsScoringDrive,PlayNumberInDrive,IsScoringPlay,PlayOutcome,PlayDescription,PlayStart
0,2023,Super Bowl,SUN,02/11,49ers,Chiefs,1ST QUARTER,2,KC,0,1,0,-3 Yard Run,(12:15) (Shotgun) I.Pacheco left guard to KC 2...,1st & 10 at KC 27
1,2023,Super Bowl,SUN,02/11,49ers,Chiefs,1ST QUARTER,2,KC,0,2,0,1 Yard Pass,(11:39) (Shotgun) P.Mahomes pass short left to...,2nd & 13 at KC 24
2,2023,Super Bowl,SUN,02/11,49ers,Chiefs,1ST QUARTER,2,KC,0,3,0,8 Yard Pass,(11:04) (Shotgun) P.Mahomes pass short right t...,3rd & 12 at KC 25
3,2023,Super Bowl,SUN,02/11,49ers,Chiefs,1ST QUARTER,2,KC,0,4,0,Punt,"(10:24) T.Townsend punts 43 yards to SF 24, Ce...",4th & 4 at KC 33
4,2023,Super Bowl,SUN,02/11,49ers,Chiefs,1ST QUARTER,4,KC,0,1,0,10 Yard Run,(6:28) (Shotgun) I.Pacheco right guard to KC 2...,1st & 10 at KC 11


# Isolating Different Plays
- The goal here is to parse out the different values for 'PlayOutcome'
  - separate pass / run / kickoff / etc.

## Parsing Plays


In [ ]:
df_2023_pass_sb = df_2023_plays_sb[df_2023_plays_sb['PlayOutcome'].str.contains('Pass')]
df_2023_run_sb = df_2023_plays_sb[df_2023_plays_sb['PlayOutcome'].str.contains('Run')]
df_2023_punt_sb = df_2023_plays_sb[df_2023_plays_sb['PlayOutcome'].str.contains('Punt')]
df_2023_sack_sb = df_2023_plays_sb[df_2023_plays_sb['PlayOutcome'].str.contains('Sack')]
df_2023_kickoff_sb = df_2023_plays_sb[df_2023_plays_sb['PlayOutcome'].str.contains('Kickoff')]
df_2023_fumble_sb = df_2023_plays_sb[df_2023_plays_sb['PlayOutcome'].str.contains('Fumble')]
df_2023_interception_sb = df_2023_plays_sb[df_2023_plays_sb['PlayOutcome'].str.contains('Interception')]
df_2023_penalty_sb = df_2023_plays_sb[df_2023_plays_sb['PlayOutcome'].str.contains('Penalty')]
df_2023_fieldgoal_sb = df_2023_plays_sb[df_2023_plays_sb['PlayOutcome'].str.contains('Field Goal')]
df_2023_touchdown_sb = df_2023_plays_sb[df_2023_plays_sb['PlayOutcome'].str.contains('Touchdown')]
df_2023_extrapoint_sb = df_2023_plays_sb[df_2023_plays_sb['PlayOutcome'].str.contains('Extra Point')]

plays_list = [df_2023_pass_sb,
              df_2023_run_sb,
              df_2023_punt_sb,
              df_2023_sack_sb,
              df_2023_kickoff_sb,
              df_2023_fumble_sb,
              df_2023_interception_sb,
              df_2023_penalty_sb,
              df_2023_fieldgoal_sb,
              df_2023_touchdown_sb,
              df_2023_extrapoint_sb]

## Sanity Check (All Plays Accounted for)

In [ ]:
df_check = pd.DataFrame()
for i in plays_list:
  df_check = pd.concat([df_check, i])

In [ ]:
df_check = df_check.sort_index()

In [ ]:
df_2023_plays_sb.equals(df_check)

True

# PlayDescription Feature Breakdown


## Pass Plays

In [ ]:
df_2023_pass_sb['PlayOutcome'].unique()

array(['1 Yard Pass', '8 Yard Pass', 'Pass for No Gain', '7 Yard Pass',
       '52 Yard Pass', 'Pass Incomplete', '10 Yard Pass', '9 Yard Pass',
       '5 Yard Pass', '18 Yard Pass', '11 Yard Pass', '12 Yard Pass',
       '-8 Yard Pass', '6 Yard Pass', '21 Yard Pass', '3 Yard Pass',
       '2 Yard Pass', '16 Yard Pass', '13 Yard Pass', '25 Yard Pass',
       '22 Yard Pass', '-3 Yard Pass', '4 Yard Pass', '19 Yard Pass',
       '17 Yard Pass', '20 Yard Pass', '23 Yard Pass', '24 Yard Pass'],
      dtype=object)

In [ ]:
# 3 different formats as far as I can see.
# 1. '# Yard Pass'
# 2. 'Pass Incomplete'
# 3. 'Pass for No Gain'

In [ ]:
df_2023_pass_sb

,Season,Week,Day,Date,AwayTeam,HomeTeam,Quarter,DriveNumber,TeamWithPossession,IsScoringDrive,PlayNumberInDrive,IsScoringPlay,PlayOutcome,PlayDescription,PlayStart
1,2023,Super Bowl,SUN,02/11,49ers,Chiefs,1ST QUARTER,2,KC,0,2,0,1 Yard Pass,(11:39) (Shotgun) P.Mahomes pass short left to...,2nd & 13 at KC 24
2,2023,Super Bowl,SUN,02/11,49ers,Chiefs,1ST QUARTER,2,KC,0,3,0,8 Yard Pass,(11:04) (Shotgun) P.Mahomes pass short right t...,3rd & 12 at KC 25
6,2023,Super Bowl,SUN,02/11,49ers,Chiefs,1ST QUARTER,4,KC,0,3,0,Pass for No Gain,(5:15) (Shotgun) P.Mahomes pass short left to ...,2nd & 14 at KC 17
10,2023,Super Bowl,SUN,02/11,49ers,Chiefs,2ND QUARTER,2,KC,0,2,0,7 Yard Pass,(14:48) P.Mahomes pass short left to I.Pacheco...,1st & 10 at KC 25
13,2023,Super Bowl,SUN,02/11,49ers,Chiefs,2ND QUARTER,2,KC,0,5,0,52 Yard Pass,(13:01) (Shotgun) P.Mahomes pass deep right to...,1st & 10 at KC 39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,2023,Super Bowl,SUN,02/11,49ers,Chiefs,OVERTIME,1,SF,1,4,0,2 Yard Pass,(14:55) (Shotgun) B.Purdy pass short middle to...,2nd & 15 at SF 20
181,2023,Super Bowl,SUN,02/11,49ers,Chiefs,OVERTIME,1,SF,1,8,0,11 Yard Pass,(12:38) (Shotgun) B.Purdy pass short left to B...,3rd & 2 at SF 35
183,2023,Super Bowl,SUN,02/11,49ers,Chiefs,OVERTIME,1,SF,1,10,0,24 Yard Pass,(11:12) (Shotgun) B.Purdy pass short left to C...,2nd & 6 at 50
186,2023,Super Bowl,SUN,02/11,49ers,Chiefs,OVERTIME,1,SF,1,13,0,13 Yard Pass,(9:25) (Shotgun) B.Purdy pass short right to K...,2nd & 12 at KC 28


In [ ]:
string = df_2023_plays['PlayDescription'].iloc[1]

In [ ]:
# Regular expression pattern
pattern = r'\b[A-Z]\.[A-Za-z]+\b'

# Search for the pattern
matches = re.findall(pattern, string)

# Output the matches
print(matches)

['J.Love', 'A.Jones', 'C.Sample']
